In [16]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import statistics

In [17]:
env = environment2.environment2()
user_list_2D = env.user_list_2D

In [18]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv'

In [19]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [20]:
fileNames2D=user_list_2D

In [21]:
fileNames2D

array(['data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_a6aab5f5-

In [22]:
def get_ROISubset(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
        if data["State"][i]=='Answering':
            data["State"][i]='Sensemaking'
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)

    replacement_dict = {(1, 2): 0, (3, 4): 1, (5, 6): 2, (7,8):3 , (9,10):4 , (11,12):5}
    combined_roi_subset = [next(v for k, v in replacement_dict.items() if i in k) for i in roi_subset]
    return combined_roi_subset



In [23]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=0


    current_state=dataframe['State'][0]
    for i in range(1,len(dataframe)):
      if dataframe["State"][i]=='Answering':
        dataframe["State"][i]='Sensemaking'
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i-1]="same"
      else:
        dataframe["high-level-action"][i-1]="change"
      current_state=dataframe["State"][i]
    dataframe["high-level-action"][len(dataframe)-1]="same"
    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])

      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [24]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None
    def Mann_Whitney_U_Test(self,timeseries_X,timeseries_Y):
        #check normality
        # result1 =shapiro(timeseries_X)
        # result2 =shapiro(timeseries_Y)
        # px=result1.pvalue
        # py=result2.pvalue

        # if px < self.SignificanceLevel and py < self.SignificanceLevel:
        #     print("X Y not normal",px,py)
        # else:
        #     print("Normal",px,py)
        #return ttest_ind(timeseries_X, timeseries_Y)

        return mannwhitneyu(timeseries_X, timeseries_Y,alternative='two-sided')

    def Wilcoxon_Test(self,timeseries_X, timeseries_Y):

        return wilcoxon(timeseries_X, timeseries_Y)

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [25]:
user=[]
roi_cycle=[]
all_states=[]
result=[]


output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','State','Result'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  states=["Sensemaking",'Foraging','Navigation']
  serieses=np.empty(shape=(len(df.ROI_Cycle.unique()),len(states)),dtype=object)
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    for j in range(len(states)):

        state_df=dfs.loc[(dfs['State'] == states[j])]
        state_df.reset_index(inplace=True)

        same_probs=[]
        for indx in range(len(state_df)):
              if state_df['high-level-action'][indx]=='same':
                  same_probs.append(state_df['probabilities'][indx])
              else:
                  same_probs.append(1-state_df['probabilities'][indx])

        serieses[k][j]=same_probs

  for id in range(serieses.shape[1]):
      combinations=[]
      for first_cycle in range(serieses.shape[0]):
          for cycle in range(serieses.shape[0]):
            if first_cycle != cycle and list([first_cycle, cycle]) not in combinations and list(reversed([first_cycle, cycle])) not in combinations:
              print("Roi Cycles: #######################",[first_cycle,cycle]," ########State######## ",states[id])
              #for final dataframe
              all_states.append(states[id])
              user.append(u)
              roi_cycle.append([first_cycle,cycle])
              combinations.append([first_cycle,cycle])
              x=list(serieses[first_cycle][id])
              y=list(serieses[cycle][id])
              print("Initial Length: ",len(x),len(y))

              if len(x) == 0:
                x = [0.5] * len(y)
              elif len(y) == 0:
                y = [0.5]* len(x)
              else:
                  print(" ")

              if  (len(x)>2 or len(y)>2):
                        sTest_state = StationarityTests()
                        trend=sTest_state.Mann_Whitney_U_Test(x, y)
                        print("State is: ", states[id], "  MWUT Result Non-Stationarity:  ", trend.pvalue < 0.05, trend.pvalue)
                        result.append(trend.pvalue < 0.05)
              else :
                         print("State is: ", states[id], "  MWUT Result is:  ","Not Enough Data")
                         result.append('NED')
            else:
                print("already calculated")

output_dataframe['User']=user
output_dataframe['Result']=result
output_dataframe['State']=all_states
output_dataframe['ROI_Cycle']=roi_cycle


########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
already calculated
Roi Cycles: ####################### [0, 1]  ########State########  Sensemaking
Initial Length:  8 2
 
State is:  Sensemaking   MWUT Result Non-Stationarity:   True 0.049491231371311464
Roi Cycles: ####################### [0, 2]  ########State########  Sensemaking
Initial Length:  8 5
 
State is:  Sensemaking   MWUT Result Non-Stationarity:   True 0.022704184640876996
Roi Cycles: ####################### [0, 3]  ########State########  Sensemaking
Initial Length:  8 0
State is:  Sensemaking   MWUT Result Non-Stationarity:   True 0.00040535351341844194
already calculated
already calculated
Roi Cycles: ####################### [1, 2]  ########State########  Sensemaking
Initial Length:  2 5
 
State is:  Sensemaking   MWUT Result Non-Stationarity:   False 0.32412658775761904
Roi Cycles: ####################### [1, 3]  ########State########  Sensemaking
Initial Lengt

In [28]:
output_dataframe.head(30)

,User,ROI_Cycle,State,Result
0,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 1]",Sensemaking,True
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 2]",Sensemaking,True
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 3]",Sensemaking,True
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[1, 2]",Sensemaking,False
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[1, 3]",Sensemaking,NED
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[2, 3]",Sensemaking,False
6,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 1]",Foraging,False
7,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 2]",Foraging,False
8,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[0, 3]",Foraging,False
9,userid_44968286-f204-4ad6-a9b5-d95b38e97866,"[1, 2]",Foraging,False


In [27]:
output_dataframe.to_csv("data/NDSI-2D\\region-manwhitneyresults.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)